This is my modified version of `run_hw1.ipynb`.

In [1]:
%cd C:\Users\DeadScholar\Programming\CS 285 Fall 2022\CS-285_fall2022\hw1

C:\Users\DeadScholar\Programming\CS 285 Fall 2022\CS-285_fall2022\hw1


In [2]:
#@title test virtual display

#@markdown If you see a video of a four-legged ant fumbling about, setup is complete!

import gym
from cs285.infrastructure.colab_utils import (
    wrap_env,
    show_video
)

env = wrap_env(gym.make("Ant-v4", render_mode='rgb_array'))

observation = env.reset()
for i in range(100):
    env.render()
    obs, rew, term, _ = env.step(env.action_space.sample() ) 
    if term:
      break;

env.close()
print('Loading video...')
show_video()

C:\Users\DeadScholar\Miniconda3\envs\cs285\lib\site-packages\gym\core.py:318: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
C:\Users\DeadScholar\Miniconda3\envs\cs285\lib\site-packages\gym\wrappers\step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.

Loading video...


## Editing Code

To edit code, click the folder icon on the left menu. Navigate to the corresponding file (`cs285_f2022/...`). Double click a file to open an editor. There is a timeout of about ~12 hours with Colab while it is active (and less if you close your browser window). We sync your edits to Google Drive so that you won't lose your work in the event of an instance timeout, but you will need to re-mount your Google Drive and re-install packages with every new instance.

## Run Behavior Cloning (Problem 1)

In [39]:
#@title imports

import os
import time
import numpy as np
from ipywidgets import FloatProgress

from cs285.infrastructure.rl_trainer import RL_Trainer
from cs285.agents.bc_agent import BCAgent
from cs285.policies.loaded_gaussian_policy import LoadedGaussianPolicy
from cs285.infrastructure.utils import MJ_ENV_KWARGS, MJ_ENV_NAMES

%reload_ext autoreload
%load_ext autoreload
%autoreload 2

import logging
import ipdb
%pdb on
from importlib import reload


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Automatic pdb calling has been turned ON


In [45]:
#@title runtime arguments

class Args:

  def __getitem__(self, key):
    return getattr(self, key)

  def __setitem__(self, key, val):
    setattr(self, key, val)

  #@markdown expert data
  expert_policy_file = 'cs285/policies/experts/Ant.pkl' #@param
  expert_data = 'cs285/expert_data/expert_data_Ant-v4.pkl' #@param
  env_name = 'Ant-v4' #@param ['Ant-v4', 'Walker2d-v4', 'HalfCheetah-v4', 'Hopper-v4']
  exp_name = 'bc_ant' #@param
  do_dagger = False #@param {type: "boolean"}
  ep_len = 1000 #@param {type: "integer"}
  save_params = False #@param {type: "boolean"}

  num_agent_train_steps_per_iter = 1000 #@param {type: "integer"})
  n_iter = 1 #@param {type: "integer"})

  #@markdown batches & buffers
  batch_size = 1000 #@param {type: "integer"})
  eval_batch_size = 1000 #@param {type: "integer"}
  train_batch_size = 100 #@param {type: "integer"}
  max_replay_buffer_size = 1000000 #@param {type: "integer"}

  #@markdown network
  n_layers = 2 #@param {type: "integer"}
  size = 64 #@param {type: "integer"}
  learning_rate = 5e-3 #@param {type: "number"}

  #@markdown logging
  video_log_freq = 5 #@param {type: "integer"}
  scalar_log_freq = 1 #@param {type: "integer"}

  #@markdown gpu & run-time settings
  no_gpu = False #@param {type: "boolean"}
  which_gpu = 0 #@param {type: "integer"}
  seed = 1 #@param {type: "integer"}

args = Args()


In [46]:
#@title define `BC_Trainer`
class BC_Trainer(object):

    def __init__(self, params):
        #######################
        ## AGENT PARAMS
        #######################

        agent_params = {
            'n_layers': params['n_layers'],
            'size': params['size'],
            'learning_rate': params['learning_rate'],
            'max_replay_buffer_size': params['max_replay_buffer_size'],
            }

        self.params = params
        self.params['agent_class'] = BCAgent ## TODO: look in here and implement this
        self.params['agent_params'] = agent_params

        self.params["env_kwargs"] = MJ_ENV_KWARGS[self.params['env_name']]

        ################
        ## RL TRAINER
        ################

        self.rl_trainer = RL_Trainer(self.params) ## TODO: look in here and implement this

        #######################
        ## LOAD EXPERT POLICY
        #######################

        print('Loading expert policy from...', self.params['expert_policy_file'])
        self.loaded_expert_policy = LoadedGaussianPolicy(self.params['expert_policy_file'])
        print('Done restoring expert policy...')

    def run_training_loop(self):

        self.rl_trainer.run_training_loop(
            n_iter=self.params['n_iter'],
            initial_expertdata=self.params['expert_data'],
            collect_policy=self.rl_trainer.agent.actor,
            eval_policy=self.rl_trainer.agent.actor,
            relabel_with_expert=self.params['do_dagger'],
            expert_policy=self.loaded_expert_policy,
        )


In [47]:
import torch
torch.cuda.is_available()


False

In [48]:
#@title create directory for logging

if args.do_dagger:
    logdir_prefix = 'q2_'  # The autograder uses the prefix `q2_`
    assert args.n_iter>1, ('DAgger needs more than 1 iteration (n_iter>1) of training, to iteratively query the expert and train (after 1st warmstarting from behavior cloning).')
else:
    logdir_prefix = 'q1_'  # The autograder uses the prefix `q1_`
    assert args.n_iter==1, ('Vanilla behavior cloning collects expert data just once (n_iter=1)')

data_path ='/content/cs285_f2022/hw1/data'
if not (os.path.exists(data_path)):
    os.makedirs(data_path)
logdir = logdir_prefix + args.exp_name + '_' + args.env_name + \
         '_' + time.strftime("%d-%m-%Y_%H-%M-%S")
logdir = os.path.join(data_path, logdir)
args['logdir'] = logdir
if not(os.path.exists(logdir)):
    os.makedirs(logdir)

In [49]:
## run training
print(args.logdir)
trainer = BC_Trainer(args)
trainer.run_training_loop()

/content/cs285_f2022/hw1/data\q1_bc_ant_Ant-v4_10-09-2022_21-34-55
########################
logging outputs to  /content/cs285_f2022/hw1/data\q1_bc_ant_Ant-v4_10-09-2022_21-34-55
########################
GPU not detected. Defaulting to CPU.
Loading expert policy from... cs285/policies/experts/Ant.pkl
obs (1, 111) (1, 111)
Done restoring expert policy...


********** Iteration 0 ************

Training agent using sampled data from replay buffer...

Beginning logging procedure...

Eval_AverageReturn : 4513.1484375
Eval_StdReturn : 0.0
Eval_MaxReturn : 4513.1484375
Eval_MinReturn : 4513.1484375
Eval_AverageEpLen : 1000.0
Train_AverageReturn : 4713.6533203125
Train_StdReturn : 12.196533203125
Train_MaxReturn : 4725.849609375
Train_MinReturn : 4701.45654296875
Train_AverageEpLen : 1000.0
Train_EnvstepsSoFar : 0
TimeSinceStart : 4.580230474472046
Training Loss : -1570.13818359375
Initial_DataCollection_AverageReturn : 4713.6533203125
Done logging...




In [28]:
#@markdown You can visualize your runs with tensorboard from within the notebook

# %load_ext tensorboard
# %tensorboard --logdir /content/cs285_f2022/hw1/data

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 312), started 0:00:35 ago. (Use '!kill 312' to kill it.)

## Running DAgger (Problem 2)
Modify the settings above:
1. check the `do_dagger` box
2. set `n_iters` to `10`
3. set `exp_name` to `dagger_ant`
and then rerun the code.

In [35]:
#@title runtime arguments

class Args:

  def __getitem__(self, key):
    return getattr(self, key)

  def __setitem__(self, key, val):
    setattr(self, key, val)

  #@markdown expert data
  expert_policy_file = 'cs285/policies/experts/Ant.pkl' #@param
  expert_data = 'cs285/expert_data/expert_data_Ant-v4.pkl' #@param
  env_name = 'Ant-v4' #@param ['Ant-v4', 'Walker2d-v4', 'HalfCheetah-v4', 'Hopper-v4']
  exp_name = 'dagger_ant' #@param
  do_dagger = True #@param {type: "boolean"}
  ep_len = 1000 #@param {type: "integer"}
  save_params = False #@param {type: "boolean"}

  num_agent_train_steps_per_iter = 1000 #@param {type: "integer"})
  n_iter = 100 #@param {type: "integer"})

  #@markdown batches & buffers
  batch_size = 1000 #@param {type: "integer"})
  eval_batch_size = 1000 #@param {type: "integer"}
  train_batch_size = 100 #@param {type: "integer"}
  max_replay_buffer_size = 1000000 #@param {type: "integer"}

  #@markdown network
  n_layers = 2 #@param {type: "integer"}
  size = 64 #@param {type: "integer"}
  learning_rate = 5e-3 #@param {type: "number"}

  #@markdown logging
  video_log_freq = 5 #@param {type: "integer"}
  scalar_log_freq = 1 #@param {type: "integer"}

  #@markdown gpu & run-time settings
  no_gpu = False #@param {type: "boolean"}
  which_gpu = 0 #@param {type: "integer"}
  seed = 1 #@param {type: "integer"}

args = Args()


In [36]:
#@title define `BC_Trainer`
class BC_Trainer(object):

    def __init__(self, params):
        #######################
        ## AGENT PARAMS
        #######################

        agent_params = {
            'n_layers': params['n_layers'],
            'size': params['size'],
            'learning_rate': params['learning_rate'],
            'max_replay_buffer_size': params['max_replay_buffer_size'],
            }

        self.params = params
        self.params['agent_class'] = BCAgent ## TODO: look in here and implement this
        self.params['agent_params'] = agent_params

        self.params["env_kwargs"] = MJ_ENV_KWARGS[self.params['env_name']]

        ################
        ## RL TRAINER
        ################

        self.rl_trainer = RL_Trainer(self.params) ## TODO: look in here and implement this

        #######################
        ## LOAD EXPERT POLICY
        #######################

        print('Loading expert policy from...', self.params['expert_policy_file'])
        self.loaded_expert_policy = LoadedGaussianPolicy(self.params['expert_policy_file'])
        print('Done restoring expert policy...')

    def run_training_loop(self):

        self.rl_trainer.run_training_loop(
            n_iter=self.params['n_iter'],
            initial_expertdata=self.params['expert_data'],
            collect_policy=self.rl_trainer.agent.actor,
            eval_policy=self.rl_trainer.agent.actor,
            relabel_with_expert=self.params['do_dagger'],
            expert_policy=self.loaded_expert_policy,
        )


In [37]:
#@title create directory for logging

if args.do_dagger:
    logdir_prefix = 'q2_'  # The autograder uses the prefix `q2_`
    assert args.n_iter>1, ('DAgger needs more than 1 iteration (n_iter>1) of training, to iteratively query the expert and train (after 1st warmstarting from behavior cloning).')
else:
    logdir_prefix = 'q1_'  # The autograder uses the prefix `q1_`
    assert args.n_iter==1, ('Vanilla behavior cloning collects expert data just once (n_iter=1)')

data_path ='/content/cs285_f2022/hw1/data'
if not (os.path.exists(data_path)):
    os.makedirs(data_path)
logdir = logdir_prefix + args.exp_name + '_' + args.env_name + \
         '_' + time.strftime("%d-%m-%Y_%H-%M-%S")
logdir = os.path.join(data_path, logdir)
args['logdir'] = logdir
if not(os.path.exists(logdir)):
    os.makedirs(logdir)

In [38]:
## run training
print(args.logdir)
trainer = BC_Trainer(args)
trainer.run_training_loop()

/content/cs285_f2022/hw1/data\q2_dagger_ant_Ant-v4_10-09-2022_21-07-37
########################
logging outputs to  /content/cs285_f2022/hw1/data\q2_dagger_ant_Ant-v4_10-09-2022_21-07-37
########################
GPU not detected. Defaulting to CPU.
Loading expert policy from... cs285/policies/experts/Ant.pkl
obs (1, 111) (1, 111)
Done restoring expert policy...


********** Iteration 0 ************

Training agent using sampled data from replay buffer...

Beginning logging procedure...

Eval_AverageReturn : 4513.1484375
Eval_StdReturn : 0.0
Eval_MaxReturn : 4513.1484375
Eval_MinReturn : 4513.1484375
Eval_AverageEpLen : 1000.0
Train_AverageReturn : 4713.6533203125
Train_StdReturn : 12.196533203125
Train_MaxReturn : 4725.849609375
Train_MinReturn : 4701.45654296875
Train_AverageEpLen : 1000.0
Train_EnvstepsSoFar : 0
TimeSinceStart : 4.900057792663574
Training Loss : -1570.13818359375
Initial_DataCollection_AverageReturn : 4713.6533203125
Done logging...




********** Iteration 1 *******

The temporary file used by moviepy cannot be deleted.
The temporary file used by moviepy cannot be deleted.


Eval_AverageReturn : 4629.98193359375
Eval_StdReturn : 0.0
Eval_MaxReturn : 4629.98193359375
Eval_MinReturn : 4629.98193359375
Eval_AverageEpLen : 1000.0
Train_AverageReturn : 4671.1337890625
Train_StdReturn : 0.0
Train_MaxReturn : 4671.1337890625
Train_MinReturn : 4671.1337890625
Train_AverageEpLen : 1000.0
Train_EnvstepsSoFar : 5000
TimeSinceStart : 40.906859397888184
Training Loss : -1729.428466796875
Initial_DataCollection_AverageReturn : 4713.6533203125
Done logging...




********** Iteration 6 ************


Relabelling collected observations with labels from an expert policy...

Training agent using sampled data from replay buffer...

Beginning logging procedure...

Eval_AverageReturn : 4559.7548828125
Eval_StdReturn : 0.0
Eval_MaxReturn : 4559.7548828125
Eval_MinReturn : 4559.7548828125
Eval_AverageEpLen : 1000.0
Train_AverageReturn : 4592.25537109375
Train_StdReturn : 0.0
Train_MaxReturn : 4592.25537109375
Train_MinReturn : 4592.25537109375
Train_AverageEpLen : 1000.0
Train_E

The temporary file used by moviepy cannot be deleted.
The temporary file used by moviepy cannot be deleted.


Eval_AverageReturn : 4853.8349609375
Eval_StdReturn : 0.0
Eval_MaxReturn : 4853.8349609375
Eval_MinReturn : 4853.8349609375
Eval_AverageEpLen : 1000.0
Train_AverageReturn : 4796.99462890625
Train_StdReturn : 0.0
Train_MaxReturn : 4796.99462890625
Train_MinReturn : 4796.99462890625
Train_AverageEpLen : 1000.0
Train_EnvstepsSoFar : 10000
TimeSinceStart : 77.4127242565155
Training Loss : -1787.4552001953125
Initial_DataCollection_AverageReturn : 4713.6533203125
Done logging...




********** Iteration 11 ************


Relabelling collected observations with labels from an expert policy...

Training agent using sampled data from replay buffer...

Beginning logging procedure...

Eval_AverageReturn : 4801.419921875
Eval_StdReturn : 0.0
Eval_MaxReturn : 4801.419921875
Eval_MinReturn : 4801.419921875
Eval_AverageEpLen : 1000.0
Train_AverageReturn : 4733.29052734375
Train_StdReturn : 0.0
Train_MaxReturn : 4733.29052734375
Train_MinReturn : 4733.29052734375
Train_AverageEpLen : 1000.0
Train_Env

...

Eval_AverageReturn : 4735.70556640625
Eval_StdReturn : 0.0
Eval_MaxReturn : 4735.70556640625
Eval_MinReturn : 4735.70556640625
Eval_AverageEpLen : 1000.0
Train_AverageReturn : 4807.693359375
Train_StdReturn : 0.0
Train_MaxReturn : 4807.693359375
Train_MinReturn : 4807.693359375
Train_AverageEpLen : 1000.0
Train_EnvstepsSoFar : 87167
TimeSinceStart : 632.7051231861115
Training Loss : -1893.3348388671875
Initial_DataCollection_AverageReturn : 4713.6533203125
Done logging...




********** Iteration 86 ************


Relabelling collected observations with labels from an expert policy...

Training agent using sampled data from replay buffer...

Beginning logging procedure...

Eval_AverageReturn : 4784.509765625
Eval_StdReturn : 0.0
Eval_MaxReturn : 4784.509765625
Eval_MinReturn : 4784.509765625
Eval_AverageEpLen : 1000.0
Train_AverageReturn : 4807.005859375
Train_StdReturn : 0.0
Train_MaxReturn : 4807.005859375
Train_MinReturn : 4807.005859375
Train_AverageEpLen : 1000.0
Train_EnvstepsSoF

The temporary file used by moviepy cannot be deleted.
The temporary file used by moviepy cannot be deleted.


Eval_AverageReturn : 4695.296875
Eval_StdReturn : 0.0
Eval_MaxReturn : 4695.296875
Eval_MinReturn : 4695.296875
Eval_AverageEpLen : 1000.0
Train_AverageReturn : 4829.6884765625
Train_StdReturn : 0.0
Train_MaxReturn : 4829.6884765625
Train_MinReturn : 4829.6884765625
Train_AverageEpLen : 1000.0
Train_EnvstepsSoFar : 92167
TimeSinceStart : 671.2824354171753
Training Loss : -1860.5225830078125
Initial_DataCollection_AverageReturn : 4713.6533203125
Done logging...




********** Iteration 91 ************


Relabelling collected observations with labels from an expert policy...

Training agent using sampled data from replay buffer...

Beginning logging procedure...

Eval_AverageReturn : 4522.4033203125
Eval_StdReturn : 0.0
Eval_MaxReturn : 4522.4033203125
Eval_MinReturn : 4522.4033203125
Eval_AverageEpLen : 1000.0
Train_AverageReturn : 4681.19677734375
Train_StdReturn : 0.0
Train_MaxReturn : 4681.19677734375
Train_MinReturn : 4681.19677734375
Train_AverageEpLen : 1000.0
Train_EnvstepsSoFar 

The temporary file used by moviepy cannot be deleted.
The temporary file used by moviepy cannot be deleted.


Eval_AverageReturn : 4888.1279296875
Eval_StdReturn : 0.0
Eval_MaxReturn : 4888.1279296875
Eval_MinReturn : 4888.1279296875
Eval_AverageEpLen : 1000.0
Train_AverageReturn : 4795.931640625
Train_StdReturn : 0.0
Train_MaxReturn : 4795.931640625
Train_MinReturn : 4795.931640625
Train_AverageEpLen : 1000.0
Train_EnvstepsSoFar : 97568
TimeSinceStart : 709.5374991893768
Training Loss : -1942.097900390625
Initial_DataCollection_AverageReturn : 4713.6533203125
Done logging...




********** Iteration 96 ************


Relabelling collected observations with labels from an expert policy...

Training agent using sampled data from replay buffer...

Beginning logging procedure...

Eval_AverageReturn : 4755.7314453125
Eval_StdReturn : 0.0
Eval_MaxReturn : 4755.7314453125
Eval_MinReturn : 4755.7314453125
Eval_AverageEpLen : 1000.0
Train_AverageReturn : 4831.9912109375
Train_StdReturn : 0.0
Train_MaxReturn : 4831.9912109375
Train_MinReturn : 4831.9912109375
Train_AverageEpLen : 1000.0
Train_EnvstepsS